# Ground-State: J1-J2 model

In this code we will use NetKet to obtain the ground state of the J1-J2 model in one-dimension with periodic boundary conditions, using a Neural Network variational wave-function. 

The Hamiltonian of the model is given by:

$$ H = \sum_{i=1}^{L} J_{1}\vec{\sigma}_{i} \cdot \vec{\sigma}_{i+1} + J_{2} \vec{\sigma}_{i} \cdot \vec{\sigma}_{i+2} $$

where the sum is over sites of the 1-D chain. Here $\vec{\sigma}=(\sigma^x,\sigma^y,\sigma^z)$ is the vector of Pauli matrices. 

In [20]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
import math

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x

In [5]:
def config(J,L):
    # Define custom graph
    edge_colors = []
    for i in range(L):
        edge_colors.append([i, (i+1)%L, 1])
        edge_colors.append([i, (i+2)%L, 2])

    # Define the netket graph object
    g = nk.graph.Graph(edges=edge_colors)
    
    #Sigma^z*Sigma^z interactions
    sigmaz = [[1, 0], [0, -1]]
    mszsz = (np.kron(sigmaz, sigmaz))

    #Exchange interactions
    exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

    bond_operator = [
        (J[0] * mszsz).tolist(),
        (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),  
        (J[1] * exchange).tolist(),
    ]

    bond_color = [1, 2, 1, 2]
    
    # Spin based Hilbert Space
    hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)
    # Custom Hamiltonian operator
    return  g, hi, nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)  

In [6]:
def st_fact(hi,L):
    sf = []
    sites = []
    structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
    for i in range(0, L):
        for j in range(0, L):
            structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L
    return structure_factor

In [11]:
def g_run(g,hi,model,N,op):
    # We shall use an exchange Sampler which preserves the global magnetization (as this is a conserved quantity in the model)
    sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)

    # Construct the variational state
    vs = nk.vqs.MCState(sa, model, n_samples=N)

    # We choose a basic, albeit important, Optimizer: the Stochastic Gradient Descent
    opt = nk.optimizer.Sgd(learning_rate=0.01)

    # Stochastic Reconfiguration
    sr = nk.optimizer.SR(diag_shift=0.01)

    # We can then specify a Variational Monte Carlo object, using the Hamiltonian, sampler and optimizers chosen.
    gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)   
    return gs

In [32]:
#### a: angle in degrees; t: number of decimal places after truncation
def j_coupl(a,t):
    theta = math.radians(a); sin = math.sin(theta);cos = math.cos(theta)
    sin_t = math.trunc(cos * 10**t) / 10**t; cos_t = math.trunc(cos * 10**t) / 10**t
    J = [sin_t,cos_t]
    return J

In [34]:
J = j_coupl(190,10)
print(J)

[-0.984807753, -0.984807753]


In [30]:
J = j_coupl(45,10)
print(J)

[0.7071067811, 0.7071067811]


In [23]:
a    = 0; theta = math.radians(a)
sin = math.sin(theta);cos = math.cos(theta)
print(sin,cos)

0.0 1.0


In [24]:
a    = 15; theta = math.radians(a)
sin = math.sin(theta);cos = math.cos(theta)
print(sin,cos)

0.25881904510252074 0.9659258262890683


In [25]:
a    = 45; theta = math.radians(a)
sin = math.sin(theta);cos = math.cos(theta)
print(sin,cos)

0.7071067811865475 0.7071067811865476


In [28]:
a      = 90; theta = math.radians(a)
sin   = math.sin(theta);cos = math.cos(theta)
n     = 10
cos_t = math.trunc(cos * 10**n) / 10**n
print(sin,cos, cos_t)

1.0 6.123233995736766e-17 0.0


In [9]:
J = [1, 0.2]; L = 14
g,hi,op = config(J,L)

In [10]:
model = FFNN()

In [14]:
structure_factor = st_fact(hi,L)

In [15]:
gs = g_run(g,hi,model,1008,op)

In [18]:
n_it = 30
# Run the optimization protocol
gs.run(out='data/log/test_' + str(n_it), n_iter=n_it, obs={'Structure Factor': structure_factor})

  0%|                                                    | 0/30 [00:00<?, ?it/s]

(JsonLog('data/log/test_30', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.009810686111450195
   	Params: 0.0068585872650146484,)

## 5) Data Visualisation

Now that we have optimized our machine to find the ground state of the $J_1-J_2$ model, let's look at what we have.
The relevant data are stored in the ".log" file while the optimized parameters are in the ".wf" file. The files are all in json format.

We shall extract the energy as well as specified observables (antiferromagnetic structure factor in our case) from the ".log" file.

In [ ]:
# Load the data from the .log file
import json

data=json.load(open("test.log"))

iters = data['Energy']['iters']
energy=data['Energy']['Mean']['real']
sf=data['Structure Factor']['Mean']['real']

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(iters, energy, color='blue', label='Energy')
ax1.set_ylabel('Energy')
ax1.set_xlabel('Iteration')
ax2 = ax1.twinx() 
ax2.plot(iters, np.array(sf), color='green', label='Structure Factor')
ax2.set_ylabel('Structure Factor')
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

Let's also compute the average of those quantities (energy and neel order) over the last 50 iterations where the optimization seems to have converged.

In [ ]:
print(r"Structure factor = {0:.3f}({1:.3f})".format(np.mean(sf[-50:]),
                                              np.std(np.array(sf[-50:]))/np.sqrt(50)))
print(r"Energy = {0:.3f}({1:.3f})".format(np.mean(energy[-50:]), np.std(energy[-50:])/(np.sqrt(50))))

## 6) Sanity Check: Exact Diagonalisation

Now that we have obtained some results using VMC, it is a good time to check the quality of our results (at least for small system sizes). For this purpose, Netket provides exact diagonalisation tools. 

In [ ]:
E_gs, ket_gs = nk.exact.lanczos_ed(op, compute_eigenvectors=True)
structure_factor_gs = (ket_gs.T.conj()@structure_factor.to_linear_operator()@ket_gs).real[0,0]

In [ ]:
E_gs, ket_gs = nk.exact.lanczos_ed(op, compute_eigenvectors=True)
structure_factor_gs = (ket_gs.T.conj()@structure_factor.to_linear_operator()@ket_gs).real[0,0]

In [ ]:
print("Exact Ground-state Structure Factor: {0:.3f}".format(structure_factor_gs))
print("Exact ground state energy = {0:.3f}".format(E_gs[0]))

So we see that the both energy and the structure factor we obtained is in agreement with the value obtained via exact diagonalisation.